In [4]:
from typing import Dict, List, Union
import os
import json
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "europe-west9",
    api_endpoint: str = "europe-west9-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    
    # Initialize client that will be used to create and send requests.
    
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if type(instances) == list else [instances]
    
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    
    parameters_dict = {}
    
    parameters = json_format.ParseDict(parameters_dict, Value())
    
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    
    print("response")
    
    print(" deployed_model_id:", response.deployed_model_id)
    
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    
    predictions = response.predictions
    
    for prediction in predictions:
        print
        print(" prediction:", list(prediction))
    
    return predictions


In [5]:
# Set the environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../vertex/helios-ecosolutions-94ab5d2658af.json"

# Load the input data from a local JSON file
with open("../vertex/solar/test_input_solar.json", "r") as file:
    instances = json.load(file)
    
predictions = predict_custom_trained_model_sample(
    project="1088859348322",
    endpoint_id="2201222278807552000",
    location="europe-west9",
    instances=instances
)

with open("../vertex/solar/predictions.json", "w") as file:
    json.dump(predictions, file,
                indent=4, sort_keys=True)





FailedPrecondition: 400 The request size (347850812 bytes) exceeds 1.500MB limit.